In [1]:
import pandas as pd
import os
import glob

#On fixe le repertoire de travail
os.chdir(r'C:\Users\50695\Perso\batchs')

#Importer tous les fichiers présent dans le repertoire 
extension = 'csv'
all_filenames =[ i for i in glob.glob('*.{}'.format(extension))]

df = pd.concat([pd.read_csv(f) for f in all_filenames])

In [2]:
df.info

<bound method DataFrame.info of      Order Number           Order Date                   Item Name  Quantity  \
0          1388.0  2015-01-10 20:12:00               Onion Chutney         1   
1          1388.0  2015-01-10 20:12:00                  Mint Sauce         1   
2          1388.0  2015-01-10 20:12:00                 Lime Pickle         1   
3          1388.0  2015-01-10 20:12:00                     Paratha         1   
4          1387.0  2015-01-10 16:55:00                 Onion Bhaji         1   
..            ...                  ...                         ...       ...   
627           NaN  2019-12-07 20:57:00  Tandoori King Prawn Garlic         1   
628           NaN  2019-12-07 17:33:00  Tandoori King Prawn Masala         1   
629           NaN  2019-12-05 17:15:00  Tandoori King Prawn Masala         1   
630           NaN  2019-12-02 18:00:00  Tandoori King Prawn Garlic         1   
631           NaN  2019-12-06 02:59:00       House white wine 75cl         1   

     Pr

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194001 entries, 0 to 631
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Order Number    74818 non-null   float64
 1   Order Date      194001 non-null  object 
 2   Item Name       194001 non-null  object 
 3   Quantity        194001 non-null  int64  
 4   Product Price   194001 non-null  float64
 5   Total products  194001 non-null  int64  
 6   Order ID        119183 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 11.8+ MB


In [4]:
df.describe()

,Order Number,Quantity,Product Price,Total products,Order ID
count,74818.000000,194001.000000,194001.000000,194001.000000,119183.000000
mean,9115.638162,1.246823,5.176943,7.124948,15185.997978
std,4052.210452,0.740706,3.259806,3.374930,6088.427753
min,630.000000,1.000000,0.500000,1.000000,2096.000000
25%,5590.000000,1.000000,2.950000,5.000000,9908.000000
50%,9103.000000,1.000000,3.950000,7.000000,15191.000000
75%,12630.000000,1.000000,8.950000,9.000000,20517.000000
max,16118.000000,51.000000,17.950000,60.000000,25583.000000


In [5]:
start_week=122
end_week=126
prefix="restaurant_1"
data_dir="../batchs"


import os

def extract(data_dir: str, prefix: str, start_week: int, end_week: int) -> pd.DataFrame:
    df=pd.DataFrame()
    
    for i in range(start_week, end_week+1):
        file_path=os.path.join(data_dir,"batchs",f'{prefix}_week_{i}.csv')
        #print(file_path)
        if os.path.isfile(file_path):
            batch=pd.read_csv(file_path)
            #print(file_path)
            df=pd.concat([df,batch],sort=True)
    return df

In [6]:
df = extract(r"C:\Users\50695\Perso","restaurant_1",1,257)
df.head()

,Item Name,Order Date,Order Number,Product Price,Quantity,Total products
0,Mango Chutney,2016-11-30 17:42:00,3849,0.5,2,24
1,Onion Chutney,2016-11-30 17:42:00,3849,0.5,1,24
2,Red Sauce,2016-11-30 17:42:00,3849,0.5,1,24
3,Onion Chutney,2016-11-30 16:53:00,3847,0.5,1,5
4,Mint Sauce,2016-12-04 18:34:00,2279,0.5,1,8


In [8]:
def clean(df: pd.DataFrame) -> pd.DataFrame :
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['order_date'] = pd.to_datetime(df['order_date'])
    df['Total_product']

In [17]:
import numpy as np
import pandas as pd
#from foodcast.infrastructure.extract import extract
#from foodcast.domain.decorators import log_return_shape


#@log_return_shape
def clean(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean a raw extract of data.
    Parameters
    ----------
    df : pd.DataFrame
        Input data to clean.
    Returns
    -------
    pd.DataFrame
        Cleaned data.
    """
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df['order_date'] = pd.to_datetime(df['order_date'])
    df = df.rename(columns={'order_number': 'order_id'})
    df = df.sort_values('order_date')
    df['total_product_price'] = df['quantity']*df['product_price']
    df['cash_in'] = df.groupby('order_id')['total_product_price'].transform(np.sum)
    df = df.drop(
        columns=['item_name', 'quantity', 'product_price',
                 'total_products', 'total_product_price'],
        errors='ignore'
    )
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df


#@log_return_shape
def merge(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Combine two different data sources into a single, consistent one.
    Parameters
    ----------
    df1 : pd.DataFrame
        First dataframe to combine. Should have an 'order_id' and an 'order_date' column.
    df2 : pd.Dataframe
        Second dataframe to combine. Should have an 'order_id' and an 'order_date' column.
    Returns
    -------
    pd.DataFrame
        Combined dataframe.
    """
    df = pd.concat([df1, df2], sort=True)
    df = df.drop(columns='order_id')
    df = df.sort_values('order_date')
    df = df.reset_index(drop=True)
    return df


#@log_return_shape
def resample(df: pd.DataFrame, freq: str = '1H') -> pd.DataFrame:
    """
    Resample a time series dataframe at a given rate (hourly rate by default).
    Parameters
    ----------
    df : pd.DataFrame
        The input dataframe. Should have a column 'order_date'.
    freq: str, optional (default: '1H')
        Sampling frequency.
    Returns
    -------
    pd.dataframe
        Resampled dataframe, with one point per hour in 'order_date'.
    """
    return df.resample('1H', on='order_date').sum().reset_index()


#@log_return_shape
def etl(data_dir: str, start_week: int, end_week: int) -> pd.DataFrame:
    """
    Load a cleaned temporal slice of data.
    Parameters
    ----------
    data_dir : str
        Data directory path.
    start_week : int
        First week number (included).
    end_week : int
        Last week number (included).
    Returns
    -------
    pd.DataFrame
        Cleaned data slice between start_week and end_week.
    """
    df1 = extract(data_dir, start_week, end_week, 'restaurant_1')
    df2 = extract(data_dir, start_week, end_week, 'restaurant_2')
    df1 = clean(df1)
    df2 = clean(df2)
    df = merge(df1, df2)
    df = resample(df)
    return df

In [18]:
df

,Item Name,Order Date,Order Number,Product Price,Quantity,Total products
0,Mango Chutney,2016-11-30 17:42:00,3849,0.50,2,24
1,Onion Chutney,2016-11-30 17:42:00,3849,0.50,1,24
2,Red Sauce,2016-11-30 17:42:00,3849,0.50,1,24
3,Onion Chutney,2016-11-30 16:53:00,3847,0.50,1,5
4,Mint Sauce,2016-12-04 18:34:00,2279,0.50,1,8
...,...,...,...,...,...,...
488,Lamb Shashlick Curry,2019-12-07 19:46:00,15751,12.95,1,5
489,Chicken Shashlick Curry,2019-12-07 17:19:00,15741,12.95,1,7
490,Hazary Lamb Chilli Garlic,2019-12-05 18:56:00,14795,12.95,1,3
491,King Prawn Shaslick,2019-12-04 19:07:00,14281,12.95,1,9
